In [107]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import openpyxl as px
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline

In [108]:
file_location = (r'/Users/szas/Documents/School/Spring 2025/Quant/VHT Holdings.xlsx')
vht_holdings = pd.read_excel(file_location)
vht_holdings = vht_holdings[['Ticker', 'Weighting']]

vht_holdings = vht_holdings.set_index('Ticker')

print(vht_holdings)



        Weighting
Ticker           
LLY        0.1095
UNH        0.0792
ABBV       0.0548
JNJ        0.0487
ABT        0.0378
...           ...
RGNX       0.0000
TRDA       0.0000
CTEV       0.0000
PLRX       0.0000
QTRX       0.0000

[407 rows x 1 columns]


In [109]:
print(vht_holdings)

        Weighting
Ticker           
LLY        0.1095
UNH        0.0792
ABBV       0.0548
JNJ        0.0487
ABT        0.0378
...           ...
RGNX       0.0000
TRDA       0.0000
CTEV       0.0000
PLRX       0.0000
QTRX       0.0000

[407 rows x 1 columns]


In [110]:
#Collecting data for the VHT ETF and its holdings
ETF = 'VHT'
vht_holdings =  [
    'LLY', 'UNH', 'ABBV', 'JNJ', 'ABT', 'MRK', 'TMO', 'ISRG', 'AMGN', 'DXY',
    'BSX', 'PFE', 'GILD', 'DHR', 'SYK', 'VRTX', 'BMY', 'MDT', 'ELV', 'CI',
    'CVS', 'MCK', 'ZTS', 'REGN', 'BDX', 'HCA', 'COR', 'EW', 'GEHC', 'VEEV',
    'ALNY', 'IDXX', 'A', 'CAH', 'RMD', 'IQV', 'HUM', 'CNC', 'DXCM', 'MTD',
    'ZBH', 'STE', 'WAT', 'BIIB', 'LH', 'DGX', 'MOH', 'PODD', 'BAX', 'NTRA',
    'COO', 'WST', 'HOLX', 'BMRN', 'ILMN', 'ITCI', 'UTHR', 'THC', 'RVTY',
    'INSM', 'RPRX', 'ALGN', 'NBIX', 'SOLV', 'CORT', 'UHS', 'EHC', 'AVTR',
    'VTRS', 'INCY', 'EXEL', 'PEN', 'TECH', 'MRNA', 'HSIC', 'CHE', 'HQY',
    'MEDP', 'HALO', 'EXAS', 'CRL', 'ENSG', 'JAZZ', 'MASI', 'DVA', 'RGEN',
    'TFX', 'LNTH', 'DOCS', 'TGTX', 'MMSI', 'GMED', 'OPCH', 'BPMC', 'GH',
    'ELAN', 'GKOS', 'HIMS', 'RVMD', 'ADMA', 'INSP', 'ALKS', 'BIO', 'IONS',
    'SRPT', 'PCVX', 'BRKR', 'MDGL', 'CYTK', 'PBH', 'BBIO', 'KRYS', 'OGN',
    'AXSM', 'ROIV', 'ITGR', 'PRGO', 'PTCT', 'CRSP', 'RDNT', 'CRNX', 'IRTC',
    'XRAY', 'PRCT', 'AMED', 'SMMT', 'NVST', 'RARE', 'ICUI', 'ACHC', 'HAE',
    'CRVL', 'CON', 'SWTX', 'VKTX', 'QDEL', 'PRVA', 'VCEL', 'VCYT', 'RNA',
    'CPRX', 'ENOV', 'PDCO', 'WAY', 'XENE', 'TMDX', 'RYTM', 'PTGX', 'TWST',
    'NUVL', 'ATRC', 'NVCR', 'LGND', 'PINC', 'ALHC', 'SUPN', 'LMAT', 'BHVN',
    'SLNO', 'FOLD', 'DNLI', 'SHC', 'PGNY', 'IART', 'ARQT', 'SEM', 'NEOG',
    'ACLX', 'ADUS', 'LIVN', 'MIRM', 'DVAX', 'UFPT', 'ACAD', 'BEAM', 'CNMD',
    'MRUS', 'AKRO', 'OMCL', 'SRRK', 'SGRY', 'ASTH', 'AZTA', 'APLS', 'TDOC',
    'VERA', 'SNDX', 'COLL', 'NVAX', 'AORT', 'CLDX', 'CERT', 'HRMY', 'NEO',
    'PCRX', 'ATEC', 'ANIP', 'CLOV', 'AUPH', 'BKD', 'USPH', 'AGIO', 'WGS',
    'AMRX', 'IRON', 'TARS', 'IDYA', 'EVH', 'AMN', 'NHC', 'KYMR', 'SDGR',
    'AHCO', 'AMPH', 'AGL', 'INVA', 'CGON', 'TVTX', 'EWTX', 'ARDX', 'BCRX',
    'RXRX', 'JANX', 'BTSG', 'MD', 'ARWR', 'PHR', 'MNKD', 'TNDM', 'IMVT',
    'SPY', 'ETNB', 'ANGO', 'SAGE', 'KNSA', 'STAA', 'EMBC', 'SLP', 'FLGT',
    'OMI', 'MNMD', 'FNA', 'SPRY', 'ACCD', 'EOLS', 'TXG', 'HSTM', 'COGT',
    'EYPT', 'CDNA', 'IRMD', 'LQDA', 'GDRX', 'QURE', 'XNCR', 'VIR', 'DYN',
    'KROS', 'VRDN', 'WVE', 'PACS', 'KIDS', 'RLAY', 'ELVN', 'PNTG', 'REPL',
    'IMNM', 'ANAB', 'SIBN', 'OPK', 'AVXL', 'AVDL', 'MLTX', 'DAWN', 'ARVN',
    'CYH', 'MLYS', 'BVS', 'PRTA', 'BFLY', 'KURA', 'EHAB', 'CSTL', 'CCRN',
    'ESTA', 'SYRE', 'ZYME', 'RXST', 'ABUS', 'OFIX', 'TMCI', 'LFST', 'HROW',
    'APGE', 'GERN', 'ALT', 'NRIX', 'MGTX', 'MYGN', 'BLFS', 'NUVB', 'PRAX',
    'PAHC', 'VREX', 'SRDX', 'TBPH', 'MLAB', 'RCUS', 'CTKB', 'GRAL', 'AVNS',
    'ADPT', 'FTRE', 'MDXG', 'IOVA', 'IBRX', 'OCUL', 'RCKT', 'NTLA', 'VNDA',
    'SVRA', 'ARCT', 'YMAB', 'NMRA', 'CRGX', 'MURA', 'MODV', 'PRME', 'TSVT',
    'PLSE', 'ENTA', 'NVRO', 'TYRA', 'LAB', 'OLMA', 'AGEN', 'IRWD', 'SLRN',
    'ALXO', 'GLUE', 'ORIC', 'EDIT', 'SIGA', 'CHRS', 'ANNX', 'CYRX', 'HCAT',
    'APLT', 'LXRX', 'DH', 'RAPP', 'HLVX', 'CABA', 'FDMT', 'MRVI', 'HUMA',
    'RPTX', 'ALLO', 'ZIMV', 'ALEC', 'AMLX', 'FATE', 'MXCT', 'VERV', 'TNGX',
    'PACB', 'BMEA', 'AVIR', 'ALDX', 'VTYX', 'OM', 'AVTE', 'STOK', 'CATX',
    'ATXS', 'SAVA', 'HRTX', 'EBS', 'AVBP', 'ZNTL', 'ERAS', 'DNTH', 'DCGO',
    'CERS', 'FGEN', 'LYEL', 'PHAT', 'MCRB', 'AMWL', 'ITOS', 'IGMS', 'KRRO',
    'ORGO', 'OSUR', 'SANA', 'ATAI', 'LUNG', 'OCGN', 'TERN', 'MGNX', 'CGEM',
    'OABI', 'RGNX', 'TRDA', 'CTEV', 'PLRX', 'QTRX'
]

end_date = datetime.now()
start_date = end_date - \
            timedelta(days=1825) # 5 years of data

ETF = yf.download(ETF, start = start_date, end = end_date, auto_adjust=False)['Adj Close']
ETF = pd.DataFrame(ETF)
ETF = ETF.dropna()
print(ETF)

vht_holdings = yf.download(vht_holdings, start = start_date, end = end_date, auto_adjust=False)['Adj Close']
vht_holdings = pd.DataFrame(vht_holdings)
print(vht_holdings)

[*********************100%***********************]  1 of 1 completed

Ticker             VHT
Date                  
2020-04-30  175.779572
2020-05-01  172.234467
2020-05-04  172.766235
2020-05-05  176.404587
2020-05-06  175.219818
...                ...
2025-04-22  245.350006
2025-04-23  246.869995
2025-04-24  250.440002
2025-04-25  251.470001
2025-04-28  252.470001

[1255 rows x 1 columns]



[*********************100%***********************]  407 of 407 completed

1 Failed download:
['DXY']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-04-29 20:51:12.222249 -> 2025-04-28 20:51:12.222249)')


Ticker               A        ABBV         ABT  ABUS       ACAD  ACCD  \
Date                                                                    
2020-04-30   74.114639   66.976074   84.281433  1.16  48.310001   NaN   
2020-05-01   71.745987   67.497536   82.231369  1.07  44.930000   NaN   
2020-05-04   72.084373   66.699043   82.213051  1.21  46.590000   NaN   
2020-05-05   73.834267   69.558960   85.141716  1.27  47.380001   NaN   
2020-05-06   75.284485   69.599716   83.357086  1.42  48.180000   NaN   
...                ...         ...         ...   ...        ...   ...   
2025-04-22  102.809998  173.779999  131.729996  3.19  14.350000   NaN   
2025-04-23  105.000000  177.050003  129.839996  3.24  14.620000   NaN   
2025-04-24  107.019997  180.369995  129.380005  3.34  14.830000   NaN   
2025-04-25  106.279999  186.059998  128.850006  3.29  14.730000   NaN   
2025-04-28  106.870003  192.339996  129.529999  3.34  14.660000   NaN   

Ticker           ACHC       ACLX       ADMA       

In [111]:
returns_etf = ETF.pct_change()*100 # Calculate daily returns and drop NaN values
returns_etf = returns_etf.dropna() # Drop NaN values
print(returns_etf)

returns_holdings = vht_holdings.pct_change()*100 # Calculate daily returns and drop NaN values
returns_holdings = returns_holdings.dropna() # Drop NaN values
print(returns_holdings)

returns_holdings = pd.DataFrame(returns_holdings)
returns_etf = pd.DataFrame(returns_etf)


Ticker           VHT
Date                
2020-05-01 -2.016790
2020-05-04  0.308747
2020-05-05  2.105939
2020-05-06 -0.671620
2020-05-07  0.149079
...              ...
2025-04-22  1.910698
2025-04-23  0.619519
2025-04-24  1.446108
2025-04-25  0.411276
2025-04-28  0.397662

[1254 rows x 1 columns]
Empty DataFrame
Columns: [A, ABBV, ABT, ABUS, ACAD, ACCD, ACHC, ACLX, ADMA, ADPT, ADUS, AGEN, AGIO, AGL, AHCO, AKRO, ALDX, ALEC, ALGN, ALHC, ALKS, ALLO, ALNY, ALT, ALXO, AMED, AMGN, AMLX, AMN, AMPH, AMRX, AMWL, ANAB, ANGO, ANIP, ANNX, AORT, APGE, APLS, APLT, ARCT, ARDX, ARQT, ARVN, ARWR, ASTH, ATAI, ATEC, ATRC, ATXS, AUPH, AVBP, AVDL, AVIR, AVNS, AVTE, AVTR, AVXL, AXSM, AZTA, BAX, BBIO, BCRX, BDX, BEAM, BFLY, BHVN, BIIB, BIO, BKD, BLFS, BMEA, BMRN, BMY, BPMC, BRKR, BSX, BTSG, BVS, CABA, CAH, CATX, CCRN, CDNA, CERS, CERT, CGEM, CGON, CHE, CHRS, CI, CLDX, CLOV, CNC, CNMD, COGT, COLL, CON, COO, COR, ...]
Index: []

[0 rows x 407 columns]


/var/folders/py/pjzjs05s7494ftly1fgt3g0c0000gn/T/ipykernel_37498/250811968.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_holdings = vht_holdings.pct_change()*100 # Calculate daily returns and drop NaN values


In [112]:
returns_etf = returns_etf.pct_change()*100 # Calculate daily returns and drop NaN values
returns_etf = returns_etf.dropna() # Drop NaN values
print(returns_etf)

returns_holdings = returns_holdings.pct_change()*100 # Calculate daily returns and drop NaN values
print(returns_holdings)

# Calculate the mean and variance of the returns
mean_returns_etf = returns_etf.mean() # Mean returns
variance_returns_etf = returns_etf.var() # Variance of returns
print(mean_returns_etf)

mean_returns_holdings = returns_holdings.mean() # Mean returns
variance_returns_holdings = returns_holdings.var() # Variance of returns
print(mean_returns_holdings)

# Calculate Annual returns
annual_returns_etf = (1 + mean_returns_etf / 100) ** 252 - 1 # Annual returns
print(annual_returns_etf)

annual_returns_holdings = (1 + mean_returns_holdings / 100) ** 252 - 1 # Annual returns
print(annual_returns_holdings)

# Calculate annual STD
annual_std_etf = ETF.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))# Annual standard deviation
print(annual_std_etf)

annual_std_holdings = vht_holdings.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))# Annual standard deviation
print(annual_std_holdings)


Ticker             VHT
Date                  
2020-05-04 -115.308833
2020-05-05  582.092052
2020-05-06 -131.891718
2020-05-07 -122.196860
2020-05-08  313.665689
...                ...
2025-04-22 -187.412593
2025-04-23  -67.576323
2025-04-24  133.424492
2025-04-25  -71.559828
2025-04-28   -3.310168

[1253 rows x 1 columns]
Empty DataFrame
Columns: [A, ABBV, ABT, ABUS, ACAD, ACCD, ACHC, ACLX, ADMA, ADPT, ADUS, AGEN, AGIO, AGL, AHCO, AKRO, ALDX, ALEC, ALGN, ALHC, ALKS, ALLO, ALNY, ALT, ALXO, AMED, AMGN, AMLX, AMN, AMPH, AMRX, AMWL, ANAB, ANGO, ANIP, ANNX, AORT, APGE, APLS, APLT, ARCT, ARDX, ARQT, ARVN, ARWR, ASTH, ATAI, ATEC, ATRC, ATXS, AUPH, AVBP, AVDL, AVIR, AVNS, AVTE, AVTR, AVXL, AXSM, AZTA, BAX, BBIO, BCRX, BDX, BEAM, BFLY, BHVN, BIIB, BIO, BKD, BLFS, BMEA, BMRN, BMY, BPMC, BRKR, BSX, BTSG, BVS, CABA, CAH, CATX, CCRN, CDNA, CERS, CERT, CGEM, CGON, CHE, CHRS, CI, CLDX, CLOV, CNC, CNMD, COGT, COLL, CON, COO, COR, ...]
Index: []

[0 rows x 407 columns]
Ticker
VHT   NaN
dtype: float64
T

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/var/folders/py/pjzjs05s7494ftly1fgt3g0c0000gn/T/ipykernel_37498/1350351020.py:28: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  annual_std_holdings = vht_holdings.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))# Annual standard deviation


In [128]:
assets = pd.concat([annual_returns_holdings, annual_std_holdings], axis=1) # Concatenate annual returns and annual STD
assets.columns = ['Returns', 'Volatility'] # Rename columns
print(assets)

# Calculate Covariance matrix
cov_matrix = vht_holdings.pct_change().apply(lambda x: np.log(1+x)).cov()*252
print(cov_matrix)

cov_matrix = returns_holdings.fillna(returns_holdings.mean())
cov_matrix = returns_holdings.cov()*252 # Covariance matrix

# Simulate portfolio
portfolio_returns = []
portfolio_volatility = []
assets_weights = ()
num_assets = len(vht_holdings.index) # Number of assets in the portfolio
num_portfolios = 5000 # Number of portfolios to simulate

print(num_assets)

                     Returns  Volatility
2020-04-30 00:00:00      0.0         NaN
2020-05-01 00:00:00      0.0         NaN
2020-05-04 00:00:00      0.0         NaN
2020-05-05 00:00:00      0.0         NaN
2020-05-06 00:00:00      0.0         NaN
...                      ...         ...
ZBH                      NaN    0.266561
ZIMV                     NaN    0.801315
ZNTL                     NaN    0.861778
ZTS                      NaN    0.259848
ZYME                     NaN    0.636364

[1662 rows x 2 columns]


/var/folders/py/pjzjs05s7494ftly1fgt3g0c0000gn/T/ipykernel_37498/4224565697.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  cov_matrix = vht_holdings.pct_change().apply(lambda x: np.log(1+x)).cov()*252


Ticker         A      ABBV       ABT      ABUS      ACAD      ACCD      ACHC  \
Ticker                                                                         
A       0.076960  0.016807  0.029894  0.043287  0.035271  0.069776  0.033978   
ABBV    0.016807  0.052078  0.017800  0.014558  0.015556  0.021044  0.020398   
ABT     0.029894  0.017800  0.051789  0.030179  0.019742  0.046019  0.022789   
ABUS    0.043287  0.014558  0.030179  0.558956  0.085464  0.130494  0.029946   
ACAD    0.035271  0.015556  0.019742  0.085464  0.397672  0.091020  0.036691   
...          ...       ...       ...       ...       ...       ...       ...   
ZBH     0.028578  0.016680  0.025163  0.021853  0.027428  0.051804  0.038895   
ZIMV    0.039956  0.008155  0.024991  0.043752  0.065146  0.107467  0.023147   
ZNTL    0.068581  0.025017  0.043088  0.133314  0.114993  0.161607  0.041329   
ZTS     0.038877  0.017139  0.028755  0.029325  0.027750  0.066192  0.028368   
ZYME    0.035550  0.002793  0.014238  0.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:552: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:2894: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:2894: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [129]:

# Get the correct number of assets from your holdings data
num_assets = len(vht_holdings)

# Verify covariance matrix matches
assert cov_matrix.shape == (num_assets, num_assets), \
    f"Covariance matrix shape {cov_matrix.shape} doesn't match assets count {num_assets}"

# Initialize storage
portfolio_returns = []
portfolio_volatility = []
assets_weights = []

# Loop through each portfolio 
for portfolio in range(num_portfolios):
    # Randomly assign normalized weights
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    
    # Store weights (now properly aligned)
    assets_weights.append(weights)
    
    # Calculate portfolio return (dimensions will now match)
    returns_holdings = np.dot(weights, annual_returns_holdings)
    portfolio_returns.append(returns_holdings)
    
    # Calculate portfolio variance
    variance = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()
    std = np.sqrt(variance)
    annual_std = std * np.sqrt(252)  # Annualize
    portfolio_volatility.append(annual_std)

# Create a DataFrame linking tickers to weights
weights_df = pd.DataFrame(
    np.array(assets_weights).T,  # Transpose to get assets as rows
    index=vht_holdings.index,    # Use tickers as index
    columns=[f'Portfolio_{i}' for i in range(num_portfolios)]
)

# Add actual fund weights for comparison
weights_df['Actual_Weighting'] = vht_holdings['Weighting']

print("First 5 tickers weight comparison:")
print(weights_df.head())

AssertionError: Covariance matrix shape (407, 407) doesn't match assets count 1255

In [114]:
# Loop through each portfolio 
for portfolio in range(num_portfolios):
    # Randomly assign weights to each asset in the portfolio
    weights = np.random.random(num_assets)
    weights /= np.sum(weights) # Normalize weights to sum to 1

    # Calculate portfolio return
    assets_weights = []
    assets_weights.append(weights)
    returns_holdings = np.dot(weights, annual_returns_holdings)

    portfolio_returns.append(returns_holdings) # Append portfolio return to list
    variance = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum() # Calculate portfolio variance
    std = np.sqrt(variance) # Calculate portfolio standard deviation
    annual_std = std * np.sqrt(252) # Annualize the standard deviation
    portfolio_volatility.append(annual_std) # Append portfolio standard deviation to list

ValueError: shapes (1255,) and (407,) not aligned: 1255 (dim 0) != 407 (dim 0)